In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image
import glob

def prepare_classification_dataset(
    source_data_path,
    output_path,
    train_ratio=0.8,
    validate_images=True
):
    """
    Prepares a dataset for YOLOv8 classification training by organizing images into the expected directory structure.
    
    Args:
        source_data_path (str): Path to your raw data
        output_path (str): Where to create the organized dataset
        train_ratio (float): Proportion of data to use for training (default 0.8)
        validate_images (bool): Whether to check if images are valid before copying
    """
    # Create main directories
    train_dir = os.path.join(output_path, 'train')
    val_dir = os.path.join(output_path, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    # Get list of classes (assuming class names are directory names)
    classes = [d for d in os.listdir(source_data_path) 
              if os.path.isdir(os.path.join(source_data_path, d))]

    for class_name in classes:
        # Create class directories in train and val
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

        # Get all images for this class
        class_path = os.path.join(source_data_path, class_name)
        images = []
        for ext in ('*.jpg', '*.jpeg', '*.png', '*.bmp', '*.webp'):
            images.extend(glob.glob(os.path.join(class_path, ext)))

        # Split into train and validation sets
        train_images, val_images = train_test_split(
            images, 
            train_size=train_ratio,
            random_state=42  # For reproducibility
        )

        # Function to safely copy images
        def copy_images(image_list, destination_dir):
            for img_path in image_list:
                if validate_images:
                    try:
                        # Attempt to open the image to verify it's valid
                        with Image.open(img_path) as img:
                            img.verify()
                    except Exception as e:
                        print(f"Skipping corrupted image {img_path}: {str(e)}")
                        continue

                # Copy the image to appropriate directory
                dest_path = os.path.join(destination_dir, class_name, os.path.basename(img_path))
                shutil.copy2(img_path, dest_path)

        # Copy images to train and validation directories
        copy_images(train_images, train_dir)
        copy_images(val_images, val_dir)

    return len(classes), classes

In [ ]:
source_path = r"D:\Kananat\TF_TMJOA_jpg_x_5px"
output_path = r"D:\Kananat\TF_TMJOA_jpg_x_5px_yolo"

num_classes, class_names = prepare_classification_dataset(
    source_path,
    output_path,
    train_ratio=0.8,
    validate_images=True
)

In [1]:
from ultralytics import YOLO

model = YOLO('yolov8m-cls.pt')  # Load the classification model

# Start training
results = model.train(
    data=r"D:\Kananat\_dataset_2d",
    epochs=100,
    imgsz=224,
    batch=32,
    device='0'  # Use '0' for first GPU, 'cpu' for CPU training
)

Ultralytics 8.3.44  Python-3.12.0 torch-2.5.1 CUDA:0 (Quadro P4000, 8192MiB)
engine\trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=D:\Kananat\_dataset_2d, epochs=100, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

train: Scanning D:\Kananat\_dataset_2d\train... 15075 images, 0 corrupt: 100%|██████████| 15075/15075 [00:04<00:00, 3378.53it/s]


train: New cache created: D:\Kananat\_dataset_2d\train.cache


val: Scanning D:\Kananat\_dataset_2d\val... 844 images, 0 corrupt: 100%|██████████| 844/844 [00:00<00:00, 1123.17it/s]

val: New cache created: D:\Kananat\_dataset_2d\val.cache


...

    100/100      1.74G    0.03943          3        224: 100%|██████████| 472/472 [05:06<00:00,  1.54it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 14/14 [00:05<00:00,  2.73it/s]

                   all      0.717          1



100 epochs completed in 5.506 hours.
Optimizer stripped from runs\classify\train2\weights\last.pt, 31.7MB
Optimizer stripped from runs\classify\train2\weights\best.pt, 31.7MB

Validating runs\classify\train2\weights\best.pt...
Ultralytics 8.3.44  Python-3.12.0 torch-2.5.1 CUDA:0 (Quadro P4000, 8192MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs
train: D:\Kananat\_dataset_2d\train... found 15075 images in 2 classes  
val: D:\Kananat\_dataset_2d\val... found 844 images in 2 classes  
test: D:\Kananat\_dataset_2d\test... found 434 images in 2 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 14/14 [00:05<00:00,  2.79it/s]


                   all      0.733          1
Speed: 0.2ms preprocess, 5.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train2


In [ ]:
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

def evaluate_binary_classifier(model, test_dir):
    all_preds = []
    all_trues = []
    
    # Process test images
    for class_path in Path(test_dir).iterdir():
        if class_path.is_dir():
            class_name = class_path.name
            class_idx = list(model.names.values()).index(class_name)
            
            for img_path in class_path.glob('*.*'):
                if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
                    results = model.predict(str(img_path))
                    pred_class = results[0].probs.top1
                    
                    all_preds.append(pred_class)
                    all_trues.append(class_idx)
    
    # Convert to numpy arrays
    y_true = np.array(all_trues)
    y_pred = np.array(all_preds)
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)  # Same as sensitivity
    f1 = f1_score(y_true, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Calculate specificity (true negative rate)
    specificity = tn / (tn + fp)
    
    # Print results
    print("\nBinary Classification Metrics:")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall (Sensitivity): {recall:.3f}")
    print(f"Specificity: {specificity:.3f}")
    print(f"F1 Score: {f1:.3f}")
    
    # Print confusion matrix
    print("\nConfusion Matrix:")
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'specificity': specificity,
        'f1': f1,
        'confusion_matrix': {
            'tn': tn, 'fp': fp,
            'fn': fn, 'tp': tp
        }
    }

# Load and evaluate model
model = YOLO(r'C:\Users\kanan\Desktop\Project_TMJOA\2D_Pipeline\YOLO\runs\classify\train\weights\best.pt')
test_dir = r"D:\Kananat\TF_TMJOA_jpg_x_5px_test"
metrics = evaluate_binary_classifier(model, test_dir)

# Optional: Save results to file
with open('binary_classification_results.txt', 'w') as f:
    f.write("Binary Classification Results\n")
    f.write("-" * 30 + "\n")
    f.write(f"Accuracy: {metrics['accuracy']:.3f}\n")
    f.write(f"Precision: {metrics['precision']:.3f}\n")
    f.write(f"Recall (Sensitivity): {metrics['recall']:.3f}\n")
    f.write(f"Specificity: {metrics['specificity']:.3f}\n")
    f.write(f"F1 Score: {metrics['f1']:.3f}\n\n")
    
    f.write("Confusion Matrix\n")
    f.write("-" * 30 + "\n")
    f.write(f"True Negatives: {metrics['confusion_matrix']['tn']}\n")
    f.write(f"False Positives: {metrics['confusion_matrix']['fp']}\n")
    f.write(f"False Negatives: {metrics['confusion_matrix']['fn']}\n")
    f.write(f"True Positives: {metrics['confusion_matrix']['tp']}\n")